# Lab 3 - Query Expansion

In [1]:
import chromadb
import tqdm
import numpy as np
from openai import OpenAI
from pypdf import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter, SentenceTransformersTokenTextSplitter
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction

In [2]:
reader = PdfReader("documents/microsoft_annual_report_2022.pdf")
pdf_texts = [p.extract_text().strip() for p in reader.pages]

# Filter the empty strings
pdf_texts = [text for text in pdf_texts if text]

print(pdf_texts[0])

1 Dear shareholders, colleagues, customers, and partners:  
We are living through a period of historic economic, societal, and geopolitical change. The world in 2022 looks nothing like 
the world in 2019. As I write this, inflation is at a 40 -year high, supply chains are stretched, and the war in Ukraine is 
ongoing. At the same time, we are entering a technological era with the potential to power awesome advancements 
across every sector of our economy and society. As the world’s largest software company, this places us at a historic 
intersection of opportunity and responsibility to the world around us.  
Our mission to empower every person and every organization on the planet to achieve more has never been more 
urgent or more necessary. For all the uncertainty in the world, one thing is clear: People and organizations in every 
industry are increasingly looking to digital technology to overcome today’s challenges and emerge stronger. And no 
company is better positioned to help th

##### The reason of first doing character split and then doing SentenceTransformersTokenTextSplitter is that ```Large documents may not fit entirely into memory. By first breaking the text into character-level chunks, you can process smaller portions at a time, reducing the memory requirements for each step.```

In [3]:
character_splitter = RecursiveCharacterTextSplitter(
    # It will split on the basis of these below characters like newline etc
    separators=["\n\n", "\n", ". ", " ", ""],
    # If after splitting at separators, it got a big length then it will break down into chunk size of 1000 characters maximum
    chunk_size=1000,
    chunk_overlap=0
)
character_split_texts = character_splitter.split_text('\n\n'.join(pdf_texts))

print(character_split_texts[10])
# print(character_split_texts[10]))
print(f"\nTotal chunks: {len(character_split_texts)}")

increased, due in large part to significant global datacenter expansions and the growth in Xbox sales and usage. Despite 
these increases, we remain dedicated to achieving a net -zero future. We recognize that progress won’t always be linear, 
and the rate at which we can implement emissions reductions is dependent on many factors that can fluctuate over time.  
On the path to becoming water positive, we invested in 21 water replenishment projects that are expected to generate 
over 1.3  million cubic meters of volumetric benefits in nine water basins around the world. Progress toward our zero waste 
commitment included diverting more than 15,200 metric tons of solid waste otherwise headed to landfills and incinerators, 
as well as launching new Circular Centers to increase reuse and reduce e -waste at our datacenters.  
We contracted to protect over 17,000 acres of land (50% more than the land we use to operate), thus achieving our

Total chunks: 347


##### ```Character Splitter is not enough due the reason that the embedder which we have to use has limited 256 characters or tokens context window```

In [4]:
# Using SentenceTransformer Splitter not Embedder but the embedder has limited context window width, it uses 256 tokens context window width. 
# So we shall make chunks of 256 tokens.
# Also beyond this 256 characters will be truncated by the embedder. We shall use these chunks and then embed them to the vector database

token_splitter = SentenceTransformersTokenTextSplitter(chunk_overlap=0, tokens_per_chunk=256) # tokens_per_chunk is context window which means that it one chunk would have 256 tokens

# We shall use all the chunks made by character text splitter and we are resplitting them using the token text splitter
token_split_texts = []
for text in character_split_texts:
    token_split_texts += token_splitter.split_text(text)

print(token_split_texts[10])
# print(token_split_texts[10]))
print(f"\nTotal chunks: {len(token_split_texts)}")

e:\My Projects\AI-ML\NLP\LLMs\RAGs\DeepLearning AI - Advanced Retrieval for AI with Chroma\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


increased, due in large part to significant global datacenter expansions and the growth in xbox sales and usage. despite these increases, we remain dedicated to achieving a net - zero future. we recognize that progress won ’ t always be linear, and the rate at which we can implement emissions reductions is dependent on many factors that can fluctuate over time. on the path to becoming water positive, we invested in 21 water replenishment projects that are expected to generate over 1. 3 million cubic meters of volumetric benefits in nine water basins around the world. progress toward our zero waste commitment included diverting more than 15, 200 metric tons of solid waste otherwise headed to landfills and incinerators, as well as launching new circular centers to increase reuse and reduce e - waste at our datacenters. we contracted to protect over 17, 000 acres of land ( 50 % more than the land we use to operate ), thus achieving our

Total chunks: 349


In [5]:
# Sentence-BERT: Sentence Transformer Embedder allows you to embed small docs, chunks etc by pooling the output
embedding_function = SentenceTransformerEmbeddingFunction()
print(embedding_function([token_split_texts[10]]))

[[0.04256269335746765, 0.03321182727813721, 0.030340097844600677, -0.0348665826022625, 0.0684165209531784, -0.08090908825397491, -0.015474393032491207, -0.0014509347965940833, -0.016744492575526237, 0.06770770251750946, -0.05054137855768204, -0.04919535666704178, 0.051399923861026764, 0.09192728996276855, -0.07177837193012238, 0.03951966390013695, -0.0128335477784276, -0.024947475641965866, -0.046228647232055664, -0.024357525631785393, 0.03394964709877968, 0.025502456352114677, 0.027317140251398087, -0.004126211628317833, -0.03633834421634674, 0.0036909214686602354, -0.027430448681116104, 0.004796730820089579, -0.02889624424278736, -0.01887071132659912, 0.03666630759835243, 0.02569584548473358, 0.03131285309791565, -0.06393438577651978, 0.05394403636455536, 0.08225348591804504, -0.04175686463713646, -0.00699579156935215, -0.023486029356718063, -0.030747944489121437, -0.002979236887767911, -0.07790941745042801, 0.009353121742606163, 0.003162869019433856, -0.02225707657635212, -0.0182946

In [6]:
chroma_client = chromadb.Client()
chroma_collection = chroma_client.create_collection("microsoft_annual_report_2022_Part4", embedding_function=embedding_function)

ids = [str(i) for i in range(len(token_split_texts))]

#---------- NEWLY ENTERED LINE: Split data into batches
batch_size = 166  # Chosen to be under the maximum allowed batch size
for i in range(0, len(token_split_texts), batch_size):
    batch_ids = ids[i:i + batch_size]
    batch_texts = token_split_texts[i:i + batch_size]
    chroma_collection.add(ids=batch_ids, documents=batch_texts)

chroma_collection.count()

349

In [7]:
def project_embeddings(embeddings, umap_transform):
    umap_embeddings = np.empty((len(embeddings),2))
    for i, embedding in enumerate(tqdm(embeddings)): 
        umap_embeddings[i] = umap_transform.transform([embedding])
    return umap_embeddings   

In [8]:
import os
import openai
from openai import OpenAI

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

openai_client = OpenAI()

In [10]:
import umap

embeddings = chroma_collection.get(include=['embeddings'])['embeddings']
umap_transform = umap.UMAP(random_state=0, transform_seed=0).fit(embeddings)
projected_dataset_embeddings = project_embeddings(embeddings, umap_transform)

AttributeError: module 'umap' has no attribute 'UMAP'

## Expansion with generated answers

https://arxiv.org/abs/2305.03653

In [ ]:
def augment_query_generated(query, model="gpt-3.5-turbo"):
    messages = [
        {
            "role": "system",
            "content": "You are a helpful expert financial research assistant. Provide an example answer to the given question, that might be found in a document like an annual report. "
        },
        {"role": "user", "content": query}
    ] 

    response = openai_client.chat.completions.create(
        model=model,
        messages=messages,
    )
    content = response.choices[0].message.content
    return content

In [ ]:
original_query = "Was there significant turnover in the executive team?"
hypothetical_answer = augment_query_generated(original_query)

joint_query = f"{original_query} {hypothetical_answer}"
print(joint_query)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
results = chroma_collection.query(query_texts=joint_query, n_results=5, include=['documents', 'embeddings'])
retrieved_documents = results['documents'][0]

for doc in retrieved_documents:
    print(doc)
    print('')

In [ ]:
retrieved_embeddings = results['embeddings'][0]
original_query_embedding = embedding_function([original_query])
augmented_query_embedding = embedding_function([joint_query])

projected_original_query_embedding = project_embeddings(original_query_embedding, umap_transform)
projected_augmented_query_embedding = project_embeddings(augmented_query_embedding, umap_transform)
projected_retrieved_embeddings = project_embeddings(retrieved_embeddings, umap_transform)

In [ ]:
import matplotlib.pyplot as plt

# Plot the projected query and retrieved documents in the embedding space
plt.figure()
plt.scatter(projected_dataset_embeddings[:, 0], projected_dataset_embeddings[:, 1], s=10, color='gray')
plt.scatter(projected_retrieved_embeddings[:, 0], projected_retrieved_embeddings[:, 1], s=100, facecolors='none', edgecolors='g')
plt.scatter(projected_original_query_embedding[:, 0], projected_original_query_embedding[:, 1], s=150, marker='X', color='r')
plt.scatter(projected_augmented_query_embedding[:, 0], projected_augmented_query_embedding[:, 1], s=150, marker='X', color='orange')

plt.gca().set_aspect('equal', 'datalim')
plt.title(f'{original_query}')
plt.axis('off')

## Expansion with multiple queries

In [ ]:
def augment_multiple_query(query, model="gpt-3.5-turbo"):
    messages = [
        {
            "role": "system",
            "content": "You are a helpful expert financial research assistant. Your users are asking questions about an annual report. "
            "Suggest up to five additional related questions to help them find the information they need, for the provided question. "
            "Suggest only short questions without compound sentences. Suggest a variety of questions that cover different aspects of the topic."
            "Make sure they are complete questions, and that they are related to the original question."
            "Output one question per line. Do not number the questions."
        },
        {"role": "user", "content": query}
    ]

    response = openai_client.chat.completions.create(
        model=model,
        messages=messages,
    )
    content = response.choices[0].message.content
    content = content.split("\n")
    return content

In [ ]:
original_query = "What were the most important factors that contributed to increases in revenue?"
augmented_queries = augment_multiple_query(original_query)

for query in augmented_queries:
    print(query)

In [ ]:
queries = [original_query] + augmented_queries
results = chroma_collection.query(query_texts=queries, n_results=5, include=['documents', 'embeddings'])

retrieved_documents = results['documents']

# Deduplicate the retrieved documents
unique_documents = set()
for documents in retrieved_documents:
    for document in documents:
        unique_documents.add(document)

for i, documents in enumerate(retrieved_documents):
    print(f"Query: {queries[i]}")
    print('')
    print("Results:")
    for doc in documents:
        print(doc)
        print('')
    print('-'*100)

In [ ]:
original_query_embedding = embedding_function([original_query])
augmented_query_embeddings = embedding_function(augmented_queries)

project_original_query = project_embeddings(original_query_embedding, umap_transform)
project_augmented_queries = project_embeddings(augmented_query_embeddings, umap_transform)


In [ ]:
result_embeddings = results['embeddings']
result_embeddings = [item for sublist in result_embeddings for item in sublist]
projected_result_embeddings = project_embeddings(result_embeddings, umap_transform)


In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.scatter(projected_dataset_embeddings[:, 0], projected_dataset_embeddings[:, 1], s=10, color='gray')
plt.scatter(project_augmented_queries[:, 0], project_augmented_queries[:, 1], s=150, marker='X', color='orange')
plt.scatter(projected_result_embeddings[:, 0], projected_result_embeddings[:, 1], s=100, facecolors='none', edgecolors='g')
plt.scatter(project_original_query[:, 0], project_original_query[:, 1], s=150, marker='X', color='r')

plt.gca().set_aspect('equal', 'datalim')
plt.title(f'{original_query}')
plt.axis('off')